# Rename all BibTeX keys using JabRef

* Use JabRef, open the bib file -> Quality -> Autogenerate BibTeX keys.
* Commit the changes.
* `git diff @~1 master > changes`
* Use Sublime to get the dict below

In [ ]:
#  Supercurrent orbitalfield
replace = {
    "Gramacy2004": "gramacy2004parameter",
    "Visvalingam1990": "visvalingam1990douglas",
    "DeRose1998": "derose1998subdivision",
    "Alliez2003": "alliez2003anisotropic",
    "Nijholt2019": "Nijholt2019a",
    "WolframResearch": "Mathematica",
    "Nijholt": "adaptive_docs",
    "Vuik2018": "vuik2018reproducing",
    "Laeven2019": "laeven2019enhanced",
    "Bommer2019": "bommer2019spin",
    "Melo2019": "melo2019supercurrent",
    "Chen2017": "chen2017intelligent",
    "Takhtaganov2018": "takhtaganov2018adaptive",
    "Emery1998": "emery1998optimal",
    "Gonnet2010": "gonnet2010increasing",
    "Galassi1996": "galassi1996gnu",
    "Klein1999": "klein1999star",
    "Berger1989": "berger1989local",
    "Berger1984": "berger1984adaptive",
    "Nijholt2016": "nijholt2016orbital",
    "Dyn1990": "dyn1990data",
    "Clenshaw1960": "clenshaw1960method",
}

In [ ]:
fname = "paper.md"

In [ ]:
with open(fname, 'r') as f:
    text = f.readlines()

In [ ]:
text = ''.join(text)

In [ ]:
for old, new in replace.items():
    text = text.replace(new, old)

print(text)

# bibtex to yaml

In [ ]:
import requests

def doi2bib(doi):
    """Return a bibTeX string of metadata for a given DOI."""
    url = "http://dx.doi.org/" + doi
    headers = {"accept": "application/x-bibtex"}
    r = requests.get(url, headers=headers)
    return r.text

## Create the yaml files and `not_on_crossref.bib` files

In [ ]:
import os
import glob

bibs = [f for f in glob.glob('*bib') if 'not_on_crossref' not in f]
bibs

Go over the above bib files and fix the DOI problems and then create the `yaml`

In [ ]:
fname = 'paper.bib'
yamls = []
folder, name = os.path.split(fname)
new = os.path.join(folder, "tmp.yaml")
yamls.append(new)
cmd = f"pandoc-citeproc --bib2yaml {fname} > {new}"
print(cmd)
os.system(cmd)

In [ ]:
import yaml

with open(new, 'r') as f:
    try:
        bibs = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
start = '@article{'
entries = {}
for d in bibs['references']:
    key = d['id']
    doi = d.get('DOI')
    if doi is None:
        bib = None
        by_hand = True
    else:
        bib = doi2bib(doi)
        if not bib.startswith("@"):
            bib = "MANUALLY_ADD"
            by_hand = True
        else:
            by_hand = False
    entries[key] = dict(doi=doi, bib=bib, by_hand=by_hand)

In [ ]:
for k, d in entries.items():
    if d['bib'] == "MANUALLY_ADD":
        print(k)

In [ ]:
for k, d in entries.items():
    if d['bib'] is None:
        print(k)

In [ ]:
entries['Nijholt2016']

In [ ]:
def replace_key(key, bib_entry):
    bib_type, *_ = bib_entry.split('{')
    _, *rest = bib_entry.split(',')
    rest = ','.join(rest)
    result = bib_type + '{' + key + ',' + rest

    # XXX: I am not sure whether these substitutions are needed.
    # the problem seemed to be the utf-8 `requests.get` encoding.
    to_replace = [("ö", r"\"{o}"), ("ü", r"\"{u}"), ("ë", r"\"{e}"), ("ï", r"\"{i}") ]
    for old, new in to_replace:
        result = result.replace(old.upper(), new.upper())
        result = result.replace(old.lower(), new.lower())

    print(result, "\n")
    return result

entries = dict(sorted(entries.items()))
for key, d in entries.items():
    if d['bib'] is not None and d['bib'].startswith("@"):
        replace_key(key, d['bib'])

In [ ]:
import parse
bib = entries["Nijholt2019"]["bib"]
bib_type, *_ = bib.split('{')
_, *rest = bib.split(',')
rest = ','.join(rest)
new_bib = bib_type + '{' + "new" + ',' + rest

In [ ]:
print(new_bib)

Try to fix the above entries with the correct DOI

In [ ]:
from toolz.dicttoolz import dissoc

with open(fname[:-3] + "yaml", 'w') as f:
    data = {k: dissoc(v, "bib") for k, v in entries.items()}
    yaml.dump(data, f)

## Go from yamls to bib files

### Check for double entries and fix them! 

In [ ]:
import os
import glob

bibs = [f for f in glob.glob('*/*yaml') if 'tmp.yaml' not in f]
bibs

In [ ]:
import yaml
mapping = {}
for fname in bibs:
    with open(fname) as f:
        mapping[fname] = yaml.safe_load(f)

In [ ]:
from collections import defaultdict
items = defaultdict(list)
for fname, info in mapping.items():
    for k, v in info.items():
        if v is not None:
            v = tuple(v.items())
        items[k].append(v)

In [ ]:
for k, v in items.items():
    if len(set(v)) >= 2:
        print(k, v)
        print()

When there are no more entries above here, go to the next step

## Combine everything into one yaml

In [ ]:
# see create_bib_file.py